In [1]:
import torch
import torchvision
import random
import torch.nn as nn
import torch.nn.functional as F
from random import gauss
from tqdm import tqdm

In [2]:
def identity(x):
    return x

In [3]:
class InputNeuron():
    def __init__(self, network):
        self.net = network # the network this belongs to
        self.a = torch.tensor(0., requires_grad = False)
        
    def __repr__(self):
        return str(self.priority) + "\t" + str(float(self.a))
        
    def forward(self):
        pass

In [4]:
class Neuron():
    def __init__(self, network, a_func = F.relu):
        self.net = network # the network this belongs to
        
        self.f = a_func
        self.a = torch.tensor(0., requires_grad = True) # The activation value of the neuron
        self.w = torch.tensor([gauss(0,0.1)],requires_grad = True ) # The weights 
        self.i = torch.tensor([0.], requires_grad = False) # The input values
        
        self.in_keys = [] # keys to grab inputs from
        self.num_connections = 0
        
    def __repr__(self):
        return   str(float(self.a)) + "\t" + str(self.in_keys)
            
    
    def forward(self):
        temp = [torch.tensor(1.)] # bias
        for key in self.in_keys:
            temp.append(self.net[key].a)
        self.i = torch.stack(temp)
        self.a = torch.dot(self.i,self.w)
        self.a = self.f(self.a)
        
    def add_connection(self,index):
        self.w.requires_grad = False
        self.w = torch.cat([self.w, torch.tensor([gauss(0,1)])])
        self.i = torch.cat([self.i, torch.tensor([0.])])
        self.w.requires_grad = True
        
        self.in_keys.append(index)
        self.num_connections += 1
        

In [5]:
class Network():
    def __init__(self,n_in,n_out,max_hidden = 5000, out_a_func = identity): 
        # self.net actually links them to their node. 
        self.max = max_hidden
        self.net = [] # Ordered in terms of priority
        self.cur = 0
        self.n_in = n_in
        self.n_out = n_out
        self.outputs = torch.empty(n_out, requires_grad=True)
        
        for i in range(n_in):
            p = i/n_in
            self.net.append(InputNeuron(self))
            
        for i in range(n_out):
            p = 9+i/n_out
            self.net.append(Neuron(self))
            
    def __getitem__(self,key):
        return self.net[key]
    
    def __len__(self):
        return len(self.net)

    def __iter__(self):
        for n in self.net:
            yield n
            
    def __call__(self,inputs):
        self.enter(inputs)
        self.forward()
        return self.outputs
    
    def parameters(self):
        params = []
        # For all non inputs, return weights
        for n in self.net[self.n_in:]:
            params.append(n.w)
        return params

    def enter(self,inputs):
        inputs = inputs.view(-1)
        # For all inputs, set to t
        for n, i in zip(self.net[:self.n_in],inputs):
            n.a = i
            
    def forward(self):
        for n in self.net:
            n.forward()
        temp = [n.a for n in self.net[-self.n_out:]]
        self.outputs = torch.stack(temp)
        [n.a.retain_grad() for n in self.net]
        
    def add_neuron(self,in_connections,out_connections):
        
        new_priority = min(out_connections + [len(self.net) - self.n_out])
        # New neurons have to before out any out connections
        
        #Move connections to account for new node in list
        for n in self.net[new_priority:]:
            for i in range(len(n.in_keys)):
                if(n.in_keys[i] > new_priority):
                    n.in_keys[i] += 1
        
        self.net.insert(new_priority, Neuron(self))
        for i in in_connections:
            self.net[new_priority].add_connection(i)
        for o in out_connections:
            self.net[o+1].add_connection(new_priority)


net = Network(800,4)
inputs = torch.randn(800)

net.add_neuron([0,0.75,0.875],[9])
net.add_neuron([0.125,0.75,0.875],[9.25])
net.add_neuron([0,0.25,0.5],[9.5])
net.add_neuron([0,0.75,0.375],[9.75])
#net[9].add_connection(0.875)

optimizer = torch.optim.SGD(net.parameters(), lr=.01)
loss_fn = nn.MSELoss()
target = torch.tensor([10.0,5.0,3.0,20.0])

for i in range(1000):
    
    inputs = torch.randn(800)
    out = net(inputs)
    loss = loss_fn(out,target)
    loss.backward()
    
    if i % 100 == 0:
        print(out)
        print(net[9].w)
    #tqdm.write(str(net[9].a.grad))
    
    optimizer.step()
    optimizer.zero_grad()

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms

In [7]:
batch_size = 32

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


In [8]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(labels)

tensor([ 0,  8,  2,  2,  1,  7,  5,  7,  1,  3,  1,  2,  9,  4,
         3,  5,  1,  0,  0,  3,  3,  9,  2,  1,  5,  9,  2,  9,
         6,  8,  5,  1])


Process Process-1:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/keogh1/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/keogh1/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/keogh1/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/keogh1/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/keogh1/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.

In [38]:
net = Network(28*28,10)
#Add random connections


In [39]:
for n in net[net.n_in:]:
    print(n.in_keys)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [40]:
for epoch in range(10):
    for i in range(5):
        out = random.choice(range(len(net)-net.n_out,len(net)))
        in_conns = []
        for j in range(20):
            in_conns.append(random.choice(range(0,len(net)-net.n_out)))
            net.add_neuron(in_conns,[out])
            
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()
    running_loss = 0
    
    for i, data in enumerate(trainloader):
        if i > 50:
            break
        images, labels = data
        temp = []
        for j in range(batch_size):
            out = net(images[j])
            out = F.softmax(out,dim=0)
            temp.append(out)
        out = torch.stack(temp)
        loss = loss_fn(out,labels)
        loss.backward()
        running_loss += loss
        if i % 10 == 9:
            print(running_loss/10)
            running_loss = 0

        optimizer.step()
        optimizer.zero_grad()
        

tensor(2.3291)
tensor(2.3029)
tensor(2.2918)
tensor(2.2861)
tensor(2.2454)


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [ ]:
correct = 0
it = iter(testloader)
for i in range(200):
    images, labels = it.next()

    imshow(torchvision.utils.make_grid(images[0]))
    out = net(images[0])
    _, index = torch.max(out,0)
    if(index == labels[0]):
        correct += 1
    print(index,labels[0])
print(correct)
